# Index files
Currently, there are some problems with compiling triads. Will have to fix.

NOTE: If a task is stopped early and restarted, resulting in multiple text files, the files should be named according to this structure:
1. **[task]-[subj]-[timepoint].txt** OR **[task]-[subj]-[timepoint]\_1.txt**
2. **[task]-[subj]-[timepoint]\_2.txt**

In [1]:
from glob import glob
from shutil import rmtree
import os
from os import remove, mkdir, utime
from os.path import join, isdir, basename

import pandas as pd
from convert_eprime import index_eprime_files
from convert_eprime.tests.utils import get_test_data_path, get_config_path

raw_data_dir = join(get_test_data_path(), 'raw_files')
param_file = join(get_config_path(), 'testing_task.json')
orged_dir = join(get_test_data_path(), 'organized_files')
csv_file = join(orged_dir, 'logger.csv')

if not isdir(orged_dir):
    mkdir(orged_dir)

In [2]:
def touch(fname):
    with open(fname, 'a'):
        utime(fname, None)

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            if not f.startswith('.'):
                print('{}{}'.format(subindent, f))

In [3]:
raw_files = sorted(glob(join(raw_data_dir, '*.*')))
list_files(raw_data_dir)

raw_files/
    StopSignal-sub001-1.edat2
    StopSignal-sub001-1.txt
    StopSignal-sub001-2.edat2
    StopSignal-sub001-2.txt
    StopSignal-sub002-1.edat2
    StopSignal-sub002-1.txt
    StopSignal-sub002-2.txt
    StopSignal-sub002-2_2.txt
    StopSignal-sub003-1.txt
    StopSignal-sub003-2.edat2
    StopSignal-sub004-1.edat2
    StopSignal-sub004-1.txt
    StopSignal-sub005-1.edat2
    StopSignal-sub005-1.txt
    StopSignal-sub005-1_2.txt


In [4]:
# Organize files. Conversion is not currently implemented.
index_eprime_files.main(raw_data_dir, csv_file, param_file)

Processing sub002- post
	Successfully organized
	Couldn't be converted.
Processing sub003- pre
	Successfully organized
	Couldn't be converted.
Processing sub002- pre
	Successfully organized
	Couldn't be converted.
Processing sub005- pre
	Successfully organized
	Couldn't be converted.
Processing sub001- pre
	Successfully organized
	Successfully converted
Processing sub001- post
	Successfully organized
	Successfully converted
Processing sub004- pre
	Successfully organized
	Successfully converted
Processing sub003- post
	Successfully organized
	Couldn't be converted.


In [5]:
# Show organized files
list_files(orged_dir)

organized_files/
    logger.csv
    sub001/
        post/
            StopSignal-sub001-2.edat2
            StopSignal-sub001-2.txt
        pre/
            StopSignal-sub001-1.edat2
            StopSignal-sub001-1.txt
    sub002/
        post/
            StopSignal-sub002-2_2.txt
        pre/
            StopSignal-sub002-1.edat2
            StopSignal-sub002-1.txt
            StopSignal-sub002-2.txt
    sub003/
        post/
            StopSignal-sub003-2.edat2
        pre/
            StopSignal-sub003-1.txt
    sub004/
        pre/
            StopSignal-sub004-1.edat2
            StopSignal-sub004-1.txt
    sub005/
        pre/
            StopSignal-sub005-1.edat2
            StopSignal-sub005-1.txt
            StopSignal-sub005-1_2.txt


In [6]:
df = pd.read_csv(csv_file)
df.sort_values(by=['Subject', 'Timepoint'],
               ascending=[True, False])

,Subject,Timepoint,Organized,Date_Organized,Organized_by,Converted,Date_Converted,Converted_by,Notes
4,sub001,pre,True,2017/08/01,PY,True,2017/08/01,PY,All good.
5,sub001,post,True,2017/08/01,PY,True,2017/08/01,PY,All good.
2,sub002,pre,True,2017/08/01,PY,False,NaN,NaN,One edat and two text files- it's a thinker.
0,sub002,post,True,2017/08/01,PY,False,NaN,NaN,One text file- must be recovered.
1,sub003,pre,True,2017/08/01,PY,False,NaN,NaN,One text file- must be recovered.
7,sub003,post,True,2017/08/01,PY,False,NaN,NaN,One edat and no text file(s)- unknown problem.
6,sub004,pre,True,2017/08/01,PY,True,2017/08/01,PY,All good.
3,sub005,pre,True,2017/08/01,PY,False,NaN,NaN,One edat and two text files- it's a thinker.


In [7]:
# Post-example cleanup
# Create the old raw files
for f in raw_files:
    touch(f)

# Remove the "done" subfolder
rmtree(join(raw_data_dir, 'done'))

# Remove the organized files
rmtree(orged_dir)
mkdir(orged_dir)